CIS 5570 Programming Assignment 2

By: Brad Byard, Anthony Lewis, and Tejas Vaity

SETUP ENVIRONMENT

In [1]:
!pip install -q pyspark
!pip install gdown

Download the files from drive

In [2]:
import gdown
matrixaddition = "https://drive.google.com/file/d/1FTAQvWD39_O7PU_qqCOggDlwPrwbfOci/view?usp=drive_link"
matrixmultiplication = "https://drive.google.com/file/d/1ZHwF0BRSIQd5K_HecAp0EmUk6YhAtzwa/view?usp=drive_link"
test1 = "https://drive.google.com/file/d/19c6pJVAA17cG3zJKFTQ3l151SunqMT9Q/view?usp=drive_link"
test2 = "https://drive.google.com/file/d/1NzyiR40Zv7M5iQJ5M2Ip8BYQc7MP2jVG/view?usp=drive_link"
test3 = "https://drive.google.com/file/d/1MVoAq-ViT3KgbH345_FKmE1nq4biTKD1/view?usp=drive_link"

In [3]:

gdown.download(matrixaddition, 'matrixaddition-W25.txt', quiet=False, fuzzy=True);
gdown.download(matrixmultiplication, 'matrixmultiplication-W25.txt', quiet=False, fuzzy=True);
gdown.download(test1, '3x3-test1.txt', quiet=False, fuzzy=True);
gdown.download(test2, '4x5-test2.txt', quiet=False, fuzzy=True);
gdown.download(test3, 'test3.txt', quiet=False, fuzzy=True);

Downloading...
From: https://drive.google.com/uc?id=1FTAQvWD39_O7PU_qqCOggDlwPrwbfOci
To: /content/matrixaddition-W25.txt
100%|██████████| 7.86M/7.86M [00:00<00:00, 168MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ZHwF0BRSIQd5K_HecAp0EmUk6YhAtzwa
To: /content/matrixmultiplication-W25.txt
100%|██████████| 926k/926k [00:00<00:00, 107MB/s]
Downloading...
From: https://drive.google.com/uc?id=19c6pJVAA17cG3zJKFTQ3l151SunqMT9Q
To: /content/3x3-test1.txt
100%|██████████| 164/164 [00:00<00:00, 521kB/s]
Downloading...
From: https://drive.google.com/uc?id=1NzyiR40Zv7M5iQJ5M2Ip8BYQc7MP2jVG
To: /content/4x5-test2.txt
100%|██████████| 414/414 [00:00<00:00, 1.51MB/s]
Downloading...
From: https://drive.google.com/uc?id=1MVoAq-ViT3KgbH345_FKmE1nq4biTKD1
To: /content/test3.txt
100%|██████████| 87.0/87.0 [00:00<00:00, 286kB/s]


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

conf = SparkConf().setAppName('SparkWordCount')
sc = SparkContext.getOrCreate(conf = conf)

sqlContext = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()


ASSIGNMENT SPECIFIC SETUP

PART 1 - A generic map reduce algorithm for matrix addition of any size

In [5]:
#Expect input such as:
#M,i,j,val or M i j val

#expect input as file, test for two different matrix sizes 3x3 and 4x5
#upload test files to projects as labeled txts below

input_file_test1 = ('/content/3x3-test1.txt')
input_file_test2 = ('/content/4x5-test2.txt')

#turn input file into an RDD of strings such as ["M, 0, 0, 5", "N, 0, 0, 6", etc.]

lines1 = sc.textFile(input_file_test1)
lines2 = sc.textFile(input_file_test2)

#------------------------MAP------------------------------------------------------
# Split each line like ['M','0','0','5']
# Convert into key/value pairs: ((row, col), value)

#MAP1 - Split each line like ['M', '0', '0', '887'] Example below
#MAP2 - 1. int(p[1]) Converts '0' to the number 0 for row index
#       2. int(p[2]) Converts '0' to the number 0 for column index
#       3. float(p[3]) Converts '887' to 887.0 for a numeric cell value
#       4. (int(p[1]), int(p[2])) converts to a tuple like (0,0)
#       5. ((int(p[1]), int(p[2])), float(p[3])) Convers to a spark Key value pair

kvpairs1 = (lines1
            .map(lambda l: l.replace(',', ' ').split())
            .map(lambda p: ((int(p[1]), int(p[2])), float(p[3]))))

kvpairs2 = (lines2
            .map(lambda l: l.replace(',', ' ').split())
            .map(lambda p: ((int(p[1]), int(p[2])), float(p[3]))))

#Test - Output looks as expected
#kvpairs1.collect()[:10]
#kvpairs2.collect()[:10]


#------------------------REDUCE----------------------------------------------------
# Add values for same (row, col)
# Creates new RDD where each key is unique and the value is sum of the matrix value at that position
matrixSum1 = kvpairs1.reduceByKey(lambda a, b: a + b)
matrixSum2 = kvpairs2.reduceByKey(lambda a, b: a + b)

#Test - Output looks as expected
#matrixSum1.collect()[:10]
#matrixSum2.collect()[:10]

#------------------------SORT-----------------------------------------------------
matrixSumSorted1 = matrixSum1.sortBy(lambda kv: (kv[0][0], kv[0][1]))
matrixSumSorted2 = matrixSum2.sortBy(lambda kv: (kv[0][0], kv[0][1]))

#Test - Output looks as expected
#matrixSumSorted1.collect()[:10]
#matrixSumSorted2.collect()[:10]

#------------------------FINAL OUTPUTS---------------------------------------------
# Flatten RDD to (i, j, val) for easy to read CSV layout
flatRDD1 = matrixSumSorted1.map(lambda kv: (kv[0][0], kv[0][1], kv[1]))
flatRDD2 = matrixSumSorted2.map(lambda kv: (kv[0][0], kv[0][1], kv[1]))

# Convert to DataFrame for CSV
matrixDF1 = sqlContext.createDataFrame(flatRDD1, ['Row', 'Col', 'Value'])
matrixDF2 = sqlContext.createDataFrame(flatRDD2, ['Row', 'Col', 'Value'])

# Save to CSV
matrixDF1.toPandas().to_csv('test1_matrixAddition.csv', index=False)
matrixDF2.toPandas().to_csv('test2_matrixAddition.csv', index=False)

# Save to TXT (Spark writes folder with part-00000)
(matrixSumSorted1
 .map(lambda kv: f"{kv[0][0]} {kv[0][1]} {kv[1]}")
 .coalesce(1)
 .saveAsTextFile("test1_matrixAddition_txt"))

(matrixSumSorted2
 .map(lambda kv: f"{kv[0][0]} {kv[0][1]} {kv[1]}")
 .coalesce(1)
 .saveAsTextFile("test2_matrixAddition_txt"))

PART 2 - Map Reduce Alogirthm to add two matrices in given text file matrixaddition-W25.txt

In [6]:
#Input the given file in Canvas for use for addition
input_file_addition = ('matrixaddition-W25.txt')

#format like
# M 1 77 731
# N 174 27 550

#turn input file into an RDD of strings such as ["M, 0, 0, 5", "N, 0, 0, 6", etc.]

lines = sc.textFile(input_file_addition)

#Test - Output looks as expected
#lines.collect()[:10]

#split lines into spark key value pairs
#output should look like ((row_index, column_index), cell_value) to prep for addition of matrices

#------------------------MAP-------------------------------------------------------------------------------------

#MAP1 - Split each line like ['M', '0', '0', '887'] Example below
#MAP2 - 1. int(p[1]) Converts '0' to the number 0 for row index
#       2. int(p[2]) Converts '0' to the number 0 for column index
#       3. float(p[3]) Converts '887' to 887.0 for a numeric cell value
#       4. (int(p[1]), int(p[2])) converts to a tuple like (0,0)
#       5. ((int(p[1]), int(p[2])), float(p[3])) Convers to a spark Key value pair

kvpairs   = (lines.map(lambda l: l.replace(',', ' ').split())
                  .map(lambda p: ((int(p[1]), int(p[2])), float(p[3]))))

#Test - Output looks as expected
#kvpairs.collect()[:10]

#------------------------REDUCE----------------------------------------------------------------------------------

#REDUCE - Add up all the values for each (row, col) key
#Creates new RDD where each key is unique and the value is sum of the matrix value at that position
matrixSum = kvpairs.reduceByKey(lambda a, b: a + b)

#Test - Output looks as expected
#matrixSum.collect()[:10]

#------------------------SORT------------------------------------------------------------------------------------

matrixSumSorted = matrixSum.sortBy(lambda kv: (kv[0][0], kv[0][1]))

#Test - Output looks as expected
#matrixSumSorted.collect()[:10]

#------------------------FINAL OUTPUT---------------------------------------------------------------------------

#flatten for CSV to (i,j,value)
flatRDD = matrixSumSorted.map(lambda kv: (kv[0][0], kv[0][1], kv[1]))

# Convert the sort into DataFrame
matrixDF = sqlContext.createDataFrame(flatRDD, ['Row', 'Col', 'Value'])

# Save to CSV
matrixDF.toPandas().to_csv('matrixAdditionFinal.csv', index=False)

# Save to TXT
(matrixSumSorted
 .map(lambda kv: f"{kv[0][0]} {kv[0][1]} {kv[1]}")
 .coalesce(1)
 .saveAsTextFile("matrixAdditionFinal.Txt"))






PART 3 - a generic map reduce algorithm for single-pass matrix multiplication

In [7]:
input_file3 = "test3.txt"  # input file (e.g., "test3.txt", "matrixmultiplication-W25 - Trimmed.txt")
lines = sc.textFile(input_file3) # turn iput file into strings RDD

In [8]:
# parse lines into (label, index, vals)
def parse_with_index(rdd):
    # keep source order to get row indices
    indexed = rdd.zipWithIndex() # (line, index)
    def parse(line, index):
        parts = line.strip().split()
        label = parts[0].upper()
        vals = list(map(int, parts[1:]))
        return (label, index, vals)
    return indexed.map(lambda x: parse(x[0], x[1]))

parsed = parse_with_index(lines)

# split M and N while preserving row order for indices
M_rows = (lines.filter(lambda s: s[:1].upper() == "M") # keep M lines in file order
               .map(lambda s: list(map(int, s.split()[1:])))
               .zipWithIndex()
               .map(lambda x: (x[1], x[0])) ) # (i, m_row)

N_rows = (lines.filter(lambda s: s[:1].upper() == "N")
               .map(lambda s: list(map(int, s.split()[1:])))
               .zipWithIndex()
               .map(lambda x: (x[1], x[0])) ) # (k, n_row)

# gather small shape metadata
I = M_rows.count()
K = len(M_rows.first()[1]) # number of columns in M
J = len(N_rows.first()[1]) # number of columns in N

# broadcast dims
I_b = sc.broadcast(I)
K_b = sc.broadcast(K)
J_b = sc.broadcast(J)

# prepare RDDs keyed by matrix coordinate:
# M elements: (i, k, a_ik)
M_entries = M_rows.flatMap(lambda iv:
    [ ( (iv[0], k), iv[1][k] ) for k in range(K_b.value) ]) \
    .map(lambda x: ('A', x[0][0], x[0][1], x[1]))  # ('A', i, k, a)

# N elements: (k, j, b_kj)
N_entries = N_rows.flatMap(lambda kv:
    [ ( (kv[0], j), kv[1][j] ) for j in range(J_b.value) ]) \
    .map(lambda x: ('B', x[0][0], x[0][1], x[1]))  # ('B', k, j, b)

# mapper (single pass): replicate to ((i,j), labelled)
# for each a_ik, test ((i,j), ('A', k, a_ik)) for all j
A_labelled = M_entries.flatMap(lambda rec:
    [ ( (rec[1], j), ('A', rec[2], rec[3]) ) for j in range(J_b.value) ])

# for each b_kj, emit ((i,j), ('B', k, b_kj)) for all i
B_labelled = N_entries.flatMap(lambda rec:
    [ ( (i, rec[2]), ('B', rec[1], rec[3]) ) for i in range(I_b.value) ])

# union and ReduceByKey over (i,j): join on k and sum a_ik * b_kj
joined = A_labelled.union(B_labelled).groupByKey()

def reduce_entry(items):
    # items: iterable of ('A' or 'B', k, val)
    A = {}
    B = {}
    for label, k, v in items:
        if label == 'A':
            A[k] = v
        else:
            B[k] = v
    total = 0
    # sum over k in intersection
    for k in set(A.keys()) & set(B.keys()):
        total += A[k] * B[k] # multiply each pair of entries for the same k
    return total

output = joined.mapValues(reduce_entry)  # ((i,j), c_ij)

# order by i then j and print or save
result_rows = (output.map(lambda ijv: (ijv[0][0], ijv[0][1], ijv[1]))
    .groupBy(lambda t: t[0]) # group by i
    .mapValues(lambda entries: [v for _, j, v in sorted(entries, key=lambda x: x[1])])
    .sortByKey()
    .values())
    # .map(lambda row: " ".join(map(str, row)))) # formatting each row (a list of numbers) for print

# print to validate the algorithm reduces
for row in result_rows.take(10):
 print(" ".join(str(x) for x in row))

173 44 86 233
4263 1074 2355 7124
661 196 382 1097


Part 4: Multiplication of two matrices of shape (128, 256) and (256, 128)

In [9]:
input_matrix_multiplication_file = "matrixmultiplication-W25.txt"  # input file matrixmultiplication-W25.txt
matrix_multi_lines = sc.textFile(input_matrix_multiplication_file) # turn input file into strings RDD

In [10]:
mm_lines = sc.textFile(input_matrix_multiplication_file)

def parse_line(line):
    parts = line.strip().split()
    matrix = parts[0].strip()
    a = int(parts[1])
    b = int(parts[2])
    val = float(parts[3])
    return (matrix, a, b, val)

data = mm_lines.map(parse_line)

def mapper(record):
    matrix, a, b, val = record
    if matrix == 'M':
        return (b, ('M', a, val))
    else:
        return (a, ('N', b, val))

mapped = data.map(mapper)
grouped = mapped.groupByKey()

def multiply_by_key(record):
    j, values = record
    M_vals, N_vals = [], []
    for tag, idx, val in values:
        if tag == 'M':
            M_vals.append((idx, val))
        else:
            N_vals.append((idx, val))

    results = []
    for i, mval in M_vals:
        for k, nval in N_vals:
            results.append(((i, k), mval * nval))
    return results


products = grouped.flatMap(multiply_by_key)
result = products.reduceByKey(lambda x, y: x + y)
formatted = result.map(lambda x: f"{x[0][0]} {x[0][1]} {x[1]}")
formatted.saveAsTextFile("matrixMultiplicationFinal.txt")

# Saving the output in CSV file
df = formatted.map(lambda line: line.split()).map(lambda parts: (int(parts[0]), int(parts[1]), float(parts[2])))
matrixDF = sqlContext.createDataFrame(df, ['Row', 'Col', 'Value'])

# Save as CSV
matrixDF.coalesce(1).write.csv("matrixMultiplicationFinal.csv", header=True, mode="overwrite")


If you want to run code again remove previous output files, cleanup directory and stop spark session

In [ ]:
# Clean up any previous Spark session to improve run speed (optional, delete this code block when not needed anymore)
# sqlContext.stop()
# sc.stop()
# del sc

In [ ]:
# import shutil
# import os

# output_dir = "matrixAdditionFinal.Txt"

# def delete_files(directories):
#   for directory_path in directories:
#     if os.path.exists(directory_path):
#       shutil.rmtree(directory_path)
#       print(f"Deleted directory: {directory_path}")
#     else:
#       print(f"Directory {directory_path} does not exist.")



# dirs_to_delete = ["matrixAdditionFinal.Txt", "test1_matrixAddition_txt", "test2_matrixAddition_txt","test3_matrixMultiplication","matrix_result","matrixMultiplicationFinal.txt"]
# delete_files(dirs_to_delete)